<div align="center">
    <div style="position: relative; max-width: 2500px; height: auto;">
        <img src="https://github.com/Kwack0202/VSCode_markdown_IMG/blob/main/earth-seen-from-space.jpg?raw=true" width="100%" style="border-radius:10px; max-height: 550px;">
        <div style="position: absolute; top: 50%; left: 50%; transform: translate(-50%, -50%); text-align: center; color: white; font-size: 60px; font-weight: bold; text-shadow: 2px 2px 4px #000;">
            <span style="padding: 10px; border-radius: 10px;">Satellite IMG Segmentation</span>
        </div>
    </div>
</div>

------------------------------------------
## Openmmlab 코드 실행 환경준비

#### **step 0.** 작업환경 확인 (C언어, 파이썬, VSC 등)
- C++이 설치되지 않으면 실행 오류남

#### **step 1.** 가상환경 생성
- conda create --name openmmlab python=3.8 -y
- conda activate openmmlab

#### **step 2.** 관련 패키지 생성 (프롬프트에서 설치)
- pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118
- pip install -U openmim
- mim install mmengine
- mim install "mmcv>=2.0.0"
- pip install wandb

#### **step 3.** MMsegmentation git 연결하기
`git bash 사용`
- **git clone -b main https://github.com/open-mmlab/mmsegmentation.git**

`anaconda prompt 사용`
- **cd C:\Users\coden\OneDrive\바탕 화면\Building\mmsegmentation**
- **pip install -v -e .** 

#### **step 4.** SegLocalVisualizer 코드 수정
    def __init__(self,
                 draw: bool = False,
                 interval: int = 50,
                 show: bool = False,
                 wait_time: float = 0.,
                 backend_args: Optional[dict] = None):
        self._visualizer: SegLocalVisualizer = SegLocalVisualizer.get_instance(name="Visualizer")
        self.interval = interval
        self.show = show

## Import

In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

2.0.0+cu118 True


In [2]:
import pandas as pd
import numpy as np
from PIL import Image

import os
import cv2
from tqdm import tqdm

import os.path as osp
import mmcv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

## MMSegmentation

In [3]:
# Check MMSegmentation installation
%cd mmsegmentation
import mmseg
print(mmseg.__version__)

c:\python_code\Segmentation_project\mmsegmentation
1.2.2


In [4]:
from mmseg.apis import MMSegInferencer
models = MMSegInferencer.list_models('mmseg')

07/16 00:50:36 - mmengine - INFO - model_name: ann_r50-d8_4xb2-40k_cityscapes-512x1024
model_name: ann_r101-d8_4xb2-40k_cityscapes-512x1024
model_name: ann_r50-d8_4xb2-40k_cityscapes-769x769
model_name: ann_r101-d8_4xb2-40k_cityscapes-769x769
model_name: ann_r50-d8_4xb2-80k_cityscapes-512x1024
model_name: ann_r101-d8_4xb2-80k_cityscapes-512x1024
model_name: ann_r50-d8_4xb2-80k_cityscapes-769x769
model_name: ann_r101-d8_4xb2-80k_cityscapes-769x769
model_name: ann_r50-d8_4xb4-80k_ade20k-512x512
model_name: ann_r101-d8_4xb4-80k_ade20k-512x512
model_name: ann_r50-d8_4xb4-160k_ade20k-512x512
model_name: ann_r101-d8_4xb4-160k_ade20k-512x512
model_name: ann_r50-d8_4xb4-20k_voc12aug-512x512
model_name: ann_r101-d8_4xb4-20k_voc12aug-512x512
model_name: ann_r50-d8_4xb4-40k_voc12aug-512x512
model_name: ann_r101-d8_4xb4-40k_voc12aug-512x512
model_name: apcnet_r50-d8_4xb2-40k_cityscapes-512x1024
model_name: apcnet_r101-d8_4xb2-40k_cityscapes-512x1024
model_name: apcnet_r50-d8_4xb2-40k_cityscapes-76

In [5]:
!mkdir checkpoints

import urllib.request

url = "https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth"
file_name = "checkpoints/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth"

urllib.request.urlretrieve(url, file_name)
print("File downloaded successfully")

File downloaded successfully


In [7]:
from mmseg.apis import init_model, inference_model, show_result_pyplot

threshold = ['(0~5]', '(0~10]', '(0~15]', '(0~20]', '(0~100]', '[0~5]', '[0~10]', '[0~15]', '[0~20]', '[0~100]']
img_path = 'C:/python_code/Segmentation_project/valid/img/TRAIN_2973.png'

for thres in threshold:
    config_file = f'./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_{thres}/pidnet-s_2xb6-120k_512x512-satellite_{thres}.py'
    checkpoint_file = f'./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_{thres}/iter_500000.pth'
    
    # build the model from a config file and a checkpoint file
    model = init_model(config_file, checkpoint_file, device='cuda:0')

    # inference on given image
    result = inference_model(model, img_path)
    
    # save the visualization result, the output image would be found at the path `work_dirs/result.png`
    vis_iamge = show_result_pyplot(model, img_path, result, out_file= f'work_dirs/{thres}_result.png')

Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~5]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~10]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~15]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~20]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~100]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_[0~5]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_[0~10]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_[0~15]/iter_500000.pth
Loads checkpoint by local backend from path: ./work_dirs/pidnet-s

In [1]:
from PIL import Image
import os

def move_and_save_image(input_path, output_folder):
    # 이미지 불러오기
    image = Image.open(input_path)

    # 원본 이미지의 크기
    width, height = image.size

    # 이동할 크기
    move_size = 512

    # 이동한 횟수 계산
    moves = width - move_size + 1

    # 출력 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 이미지 이동 및 저장
    for i in range(moves):
        # 이미지를 이동
        cropped_image = image.crop((i, 0, i + move_size, height))

        # 저장 경로 설정
        output_path = os.path.join(output_folder, f"{i}.png")

        # 이미지 저장
        cropped_image.save(output_path)

if __name__ == "__main__":
    # 입력 이미지 경로 및 출력 폴더 설정
    input_image_path = "./LA.png"
    output_folder_path = "./LA/"

    # 이미지 이동 및 저장
    move_and_save_image(input_image_path, output_folder_path)

In [17]:
config_file = './work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~20]/pidnet-s_2xb6-120k_512x512-satellite_(0~20].py'
checkpoint_file = './work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~20]/iter_500000.pth'

model = init_model(config_file, checkpoint_file, device='cuda:0')

img_folder = 'C:/Users/coden/OneDrive/바탕 화면/Building/LA/'
file_list = os.listdir(img_folder)

for img in tqdm(file_list):
    
    img_path = f'C:/Users/coden/OneDrive/바탕 화면/Building/LA/{img}'
    
    result = inference_model(model, img_path)
    
    vis_iamge = show_result_pyplot(model, img_path, result, out_file= f'work_dirs/VIS/{img}')

Loads checkpoint by local backend from path: ./work_dirs/pidnet-s_2xb6-120k_512x512-satellite_(0~20]/iter_500000.pth


100%|██████████| 864/864 [10:39<00:00,  1.35it/s]


In [11]:
import os
import cv2
import numpy as np
import re

# 이미지 파일 경로
image_folder = './mmsegmentation/work_dirs/VIS/'
video_name = './PID-Net.mp4'  # 출력 영상 파일 이름

# 이미지 파일 목록 가져오기
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
# 숫자 순서대로 정렬
images.sort(key=lambda x: int(re.sub(r'\D', '', x)))

frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# 비디오 작성자 생성
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 30, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()